## Set-Up of AutoGen, get APIs, necessary imports

In [11]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print('Groq API Key loaded successfully')
else:
    print('Groq API Key loading failed, please make sure the .env file exists and the spelling is correct')

if os.getenv("TAVILY_API_KEY"):
    print('Tavily API Key loaded successfully')
else:
    print('Tavily API Key loading failed, please make sure the .env file exists and the spelling is correct')

Groq API Key loaded successfully
Tavily API Key loaded successfully


In [12]:
import re
import httpx
import os

from langchain_groq import ChatGroq
load_dotenv()

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [13]:

# Import Autogen components
from autogen import AssistantAgent, UserProxyAgent,  ConversableAgent,register_function
from typing import TypedDict, List, Annotated
import operator
from typing import TypedDict, List, Tuple
import operator

In [14]:
llm_config = {"model": "llama-3.1-70b-versatile", "api_key": os.getenv("GROQ_API_KEY"), "api_type": "groq"}
assistant = ConversableAgent("chatbot", llm_config=llm_config)

In [15]:
import os
import autogen

player_config_list = [
    {
        "model": "llama-3.1-70b-versatile",
        "api_key": os.environ.get("GROQ_API_KEY"),
        "api_type": "groq"
    },
]

## Setup the players

In [16]:
red_spymaster = autogen.AssistantAgent(
    name="Red Spymaster",
    system_message="You are the Red Spymaster in a game of Codenames. Your role is to give clues to your team.",
    llm_config={"config_list": player_config_list},
)
blue_spymaster = autogen.AssistantAgent(
    name="Blue Spymaster",
    system_message="You are the Blue Spymaster in a game of Codenames. Your role is to give clues to your team.",
    llm_config={"config_list": player_config_list},
)
red_field_agent = autogen.AssistantAgent(
    name="Red Field Agent",
    system_message="You are the Red Field Agent in a game of Codenames. Your role is to guess words based on clues from your Spymaster given a set of words.",
    llm_config={"config_list": player_config_list},
)
blue_field_agent = autogen.AssistantAgent(
    name= "Blue Field Agent",
    system_message= "You are the Blue Field agent in the game of Codenames. Your role is to guess words based on clues from your Spymaster given a set of words.",
    llm_config= {"config_list": player_config_list}
)

## Setup Proxy-Agent
- not sure if we need to


In [17]:
game_master = autogen.AssistantAgent(
    name="Game Master",
    system_message="You are the Game Master in a game of Codenames. Your role is to manage the game state and enforce rules.",
    llm_config={"config_list": player_config_list},
)

## Game Setup


In [18]:
import random

class CodenamesGame:
    def __init__(self, starting_team="red"):
        current_dir = os.getcwd()
        wordlist_path = os.path.join(current_dir, 'wordlist-eng.txt')
        
        with open(wordlist_path, 'r') as file:
            all_words = [word.strip() for word in file.readlines()]
        
        self.words = random.sample(all_words, 25)
        self.colors = ["red"] * 8 + ["blue"] * 8 + ["neutral"] * 8 + ["black"] * 2
        random.shuffle(self.colors)
        self.board = dict(zip(self.words, self.colors))
        self.guessed_words = set()
        self.current_team = starting_team
        self.game_over = False
        self.winner = None
        self.turn_ended = False

    def switch_team(self):
        self.current_team = "blue" if self.current_team == "red" else "red"
        self.turn_ended = False

    def get_board_state(self, for_spymaster=False):
        if for_spymaster:
            return self.board
        else:
            return {word: color if word in self.guessed_words else "unknown" for word, color in self.board.items()}

    def make_guess(self, word):
        if self.game_over or word not in self.words or word in self.guessed_words:
            return False, "Invalid guess"
        
        self.guessed_words.add(word)
        color = self.board[word]
        
        if color == "black":
            self.game_over = True
            self.winner = "blue" if self.current_team == "red" else "red"
            return False, f"Game over! {self.current_team.capitalize()} team guessed the black word. {self.winner.capitalize()} team wins!"
        elif color == self.current_team:
            if self.check_win_condition():
                self.game_over = True
                self.winner = self.current_team
                return True, f"Correct! The word '{word}' belongs to the {self.current_team} team. {self.winner.capitalize()} team wins!"
            return True, f"Correct! The word '{word}' belongs to the {self.current_team} team."
        else:
            self.turn_ended = True
            if color == "neutral":
                return False, f"Neutral word. Turn ends for the {self.current_team} team."
            else:
                if self.check_win_condition():
                    self.game_over = True
                    self.winner = "blue" if self.current_team == "red" else "red"
                    return False, f"Incorrect. The word '{word}' belongs to the other team. {self.winner.capitalize()} team wins!"
                return False, f"Incorrect. The word '{word}' belongs to the other team. Turn ends for the {self.current_team} team."

    def check_win_condition(self):
        red_words = sum(1 for color in self.board.values() if color == "red")
        blue_words = sum(1 for color in self.board.values() if color == "blue")
        red_guessed = sum(1 for word in self.guessed_words if self.board[word] == "red")
        blue_guessed = sum(1 for word in self.guessed_words if self.board[word] == "blue")
        
        if red_guessed == red_words:
            self.winner = "red"
            return True
        elif blue_guessed == blue_words:
            self.winner = "blue"
            return True
        return False

    def is_game_over(self):
        return self.game_over

    def get_winner(self):
        return self.winner

    def is_turn_ended(self):
        return self.turn_ended

In [19]:
# spymaster and field agent functions
def spymaster_function(game_state, team, spymaster_agent):
    available_words = [word for word, color in game_state.items() if color == team]
    if not available_words:
        return "pass", 0
    
    prompt = f"""
    You are the {team} Spymaster in a game of Codenames. Your task is to 
    give a one-word clue and a number.
    The number represents how many words on the board are related to your clue.
    Your team's words are: {', '.join(available_words)}
    Provide a clue in the format: 'clue_word, number'
    Make sure your clue is not any of the words on the board, and follows Codenames rules.
    """

    try:
        response = spymaster_agent.generate_reply(messages=[{"role": "user", "content": prompt}])
        print(f"Raw response from spymaster: {response}")  # Debug print
        
        if isinstance(response, dict) and 'content' in response:
            response_content = response['content']
        else:
            response_content = str(response)
        
        parts = response_content.split(',')
        if len(parts) != 2:
            raise ValueError(f"Unexpected response format: {response_content}")
        
        clue_word = parts[0].strip()
        number = int(parts[1].strip())
        
        return clue_word, number
    except Exception as e:
        print(f"Error in spymaster_function: {str(e)}")
        return "error", 1

def field_agent_function(game_state, clue, number, field_agent):
    available_words = [word for word, color in game_state.items() if color == "unknown"]
    prompt = f"""
    You are a Field agent in a game of Codenames. Your Spymaster has given 
    the clue "{clue}" for {number} words. The available words on the board game are:
    {', '.join(available_words)} Based on the clue, suggest up to {number} words that you think are 
    related to the clue. Provide your guesses as a comma-separated list.
    """
    response = field_agent.generate_reply(messages=[{"role": "user", "content": prompt}])
    
    print(f"Raw response from field agent: {response}")  # Debug print
    
    if isinstance(response, dict) and 'content' in response:
        response_content = response['content']
    else:
        response_content = str(response)  # Fallback to string representation if not a dict
    
    guesses = [word.strip() for word in response_content.split(',') if word.strip() in available_words]
    guesses = guesses[:number]
    return guesses

In [20]:
def play_game():
    game = CodenamesGame()
    current_team = "red"
    
    while not game.is_game_over():
        print(f"\n--- Current Team: {current_team.capitalize()} ---")
        
        spymaster_state = game.get_board_state(for_spymaster=True)
        field_agent_state = game.get_board_state(for_spymaster=False)
        
        spymaster = red_spymaster if current_team == "red" else blue_spymaster
        clue, number = spymaster_function(spymaster_state, current_team, spymaster)
        print(f"{current_team.capitalize()} Spymaster's clue: {clue}, {number}")
        
        field_agent = red_field_agent if current_team == "red" else blue_field_agent
        guesses = field_agent_function(field_agent_state, clue, number, field_agent)
        print(f"{current_team.capitalize()} Field Agent's guesses: {', '.join(guesses)}")
        
        for guess in guesses:
            success, message = game.make_guess(guess)
            print(message)
            if not success or game.is_turn_ended():
                break
        
        if game.is_game_over():
            winner = game.get_winner()
            print(f"The {winner} team wins!")
            break
        
        game.switch_team()
        current_team = game.current_team

if __name__ == "__main__":
    play_game()


--- Current Team: Red ---


/Users/elisarogers/opt/anaconda3/envs/llm-env/lib/python3.12/site-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Raw response from spymaster: {'content': 'LABOR, 3', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}
Red Spymaster's clue: LABOR, 3
Raw response from field agent: {'content': 'Based on the clue "LABOR" from the Spymaster, my guesses for the 3 words are: UNION actions aren\'t part of the clue, but words in here are things that might be \'worked\' for instance by Labor or seen in the workplace/ area an official term for an area is \'STADIUM\' and that means lots people are working and an event laborer would make at \'temple construction sites\' to get their \'strike\' through, thus are MINE and hence words that I associate with it are: MINE, STADIUM, STRIKE', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}
Red Field Agent's guesses: STADIUM, STRIKE
Incorrect. The word 'STADIUM' belongs to the other team. Turn ends for the red team.

--- Current Team: Blue ---
Raw response from spymaster: {'con